Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [45]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  regularization = 0.01 * tf.nn.l2_loss(weights)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + regularization)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [46]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 45.904358
Minibatch accuracy: 11.7%
Validation accuracy: 14.3%
Minibatch loss at step 500: 0.857892
Minibatch accuracy: 82.0%
Validation accuracy: 81.0%
Minibatch loss at step 1000: 0.688485
Minibatch accuracy: 82.0%
Validation accuracy: 81.5%
Minibatch loss at step 1500: 0.862698
Minibatch accuracy: 79.7%
Validation accuracy: 81.7%
Minibatch loss at step 2000: 0.752143
Minibatch accuracy: 81.2%
Validation accuracy: 81.8%
Minibatch loss at step 2500: 0.679517
Minibatch accuracy: 85.2%
Validation accuracy: 81.5%
Minibatch loss at step 3000: 0.837267
Minibatch accuracy: 77.3%
Validation accuracy: 82.4%
Test accuracy: 88.5%


In [199]:
batch_size = 128
hidden_layer_size = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables. 
  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
  biases_1 = tf.Variable(tf.zeros([hidden_layer_size]))
  weights_2 = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
  logits = tf.matmul(hidden_layer, weights_2) + biases_2
  regu = 0.01 * tf.nn.l2_loss(weights_1)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + regu)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_hidden = tf.nn.relu( tf.matmul(tf_valid_dataset, weights_1) + biases_1)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden, weights_2) + biases_2)
  test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
  test_prediction = tf.nn.softmax(tf.matmul(test_hidden, weights_2) + biases_2)

In [200]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3481.220947
Minibatch accuracy: 7.0%
Validation accuracy: 18.2%
Minibatch loss at step 500: 1886.183594
Minibatch accuracy: 80.5%
Validation accuracy: 79.3%
Minibatch loss at step 1000: 1147.780396
Minibatch accuracy: 82.0%
Validation accuracy: 81.4%
Minibatch loss at step 1500: 695.706360
Minibatch accuracy: 83.6%
Validation accuracy: 82.7%
Minibatch loss at step 2000: 421.112549
Minibatch accuracy: 85.9%
Validation accuracy: 82.3%
Minibatch loss at step 2500: 254.806595
Minibatch accuracy: 86.7%
Validation accuracy: 83.6%
Minibatch loss at step 3000: 155.261398
Minibatch accuracy: 83.6%
Validation accuracy: 84.0%
Test accuracy: 90.2%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [201]:
num_steps = 3001
mini_train_size = batch_size * 3
mini_train_dataset = train_dataset[0:mini_train_size, :]
mini_train_labels = train_labels[0:mini_train_size, :]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (mini_train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = mini_train_dataset[offset:(offset + batch_size), :]
    batch_labels = mini_train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3472.849609
Minibatch accuracy: 3.1%
Validation accuracy: 22.9%
Minibatch loss at step 500: 1879.315918
Minibatch accuracy: 100.0%
Validation accuracy: 58.3%
Minibatch loss at step 1000: 1139.720459
Minibatch accuracy: 100.0%
Validation accuracy: 58.3%
Minibatch loss at step 1500: 691.188721
Minibatch accuracy: 100.0%
Validation accuracy: 58.3%
Minibatch loss at step 2000: 419.174805
Minibatch accuracy: 100.0%
Validation accuracy: 58.4%
Minibatch loss at step 2500: 254.210480
Minibatch accuracy: 100.0%
Validation accuracy: 58.4%
Minibatch loss at step 3000: 154.167297
Minibatch accuracy: 100.0%
Validation accuracy: 58.4%
Test accuracy: 64.4%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [202]:
batch_size = 128
hidden_layer_size = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables. 
  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
  biases_1 = tf.Variable(tf.zeros([hidden_layer_size]))
  weights_2 = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden_layer = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1), 0.3)
  logits = tf.matmul(hidden_layer, weights_2) + biases_2
  regu = 0.01 * tf.nn.l2_loss(weights_1)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + regu)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_hidden = tf.nn.relu( tf.matmul(tf_valid_dataset, weights_1) + biases_1)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden, weights_2) + biases_2)
  test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
  test_prediction = tf.nn.softmax(tf.matmul(test_hidden, weights_2) + biases_2)

In [203]:
num_steps = 3001
mini_train_size = batch_size * 3
mini_train_dataset = train_dataset[0:mini_train_size, :]
mini_train_labels = train_labels[0:mini_train_size, :]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (mini_train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = mini_train_dataset[offset:(offset + batch_size), :]
    batch_labels = mini_train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3687.242676
Minibatch accuracy: 10.2%
Validation accuracy: 27.7%
Minibatch loss at step 500: 32.777626
Minibatch accuracy: 96.1%
Validation accuracy: 75.4%
Minibatch loss at step 1000: 11.605771
Minibatch accuracy: 96.1%
Validation accuracy: 72.7%
Minibatch loss at step 1500: 7.640301
Minibatch accuracy: 96.1%
Validation accuracy: 75.0%
Minibatch loss at step 2000: 6.254592
Minibatch accuracy: 97.7%
Validation accuracy: 74.8%
Minibatch loss at step 2500: 6.248130
Minibatch accuracy: 96.9%
Validation accuracy: 74.1%
Minibatch loss at step 3000: 4.948800
Minibatch accuracy: 96.1%
Validation accuracy: 74.7%
Test accuracy: 81.3%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [186]:
import math as math

batch_size = 128
hidden_layer1_size = 1024
hidden_layer2_size = 512
hidden_layer3_size = 256
hidden_layer4_size = 64

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables. 
  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer1_size], stddev = math.sqrt(2.0/image_size*image_size)))
  biases_1 = tf.Variable(tf.zeros([hidden_layer1_size]))
  weights_2 = tf.Variable(tf.truncated_normal([hidden_layer1_size, hidden_layer2_size], stddev = math.sqrt(2.0/hidden_layer1_size)))
  biases_2 = tf.Variable(tf.zeros([hidden_layer2_size]))
  weights_3 = tf.Variable(tf.truncated_normal([hidden_layer2_size, hidden_layer3_size], stddev = math.sqrt(2.0/hidden_layer2_size)))
  biases_3 = tf.Variable(tf.zeros([hidden_layer3_size]))
  weights_4 = tf.Variable(tf.truncated_normal([hidden_layer3_size, hidden_layer4_size], stddev = math.sqrt(2.0/hidden_layer3_size)))
  biases_4 = tf.Variable(tf.zeros([hidden_layer4_size]))
  weights_o = tf.Variable(tf.truncated_normal([hidden_layer4_size, num_labels], stddev = math.sqrt(2.0/hidden_layer4_size)))
  biases_o = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  drop_keep = 1.0
  hidden_layer1 = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1),drop_keep)
  hidden_layer2 = tf.nn.dropout(tf.nn.relu(tf.matmul(hidden_layer1, weights_2) + biases_2),drop_keep)
  hidden_layer3 = tf.nn.dropout(tf.nn.relu(tf.matmul(hidden_layer2, weights_3) + biases_3),drop_keep)
  hidden_layer4 = tf.nn.dropout(tf.nn.relu(tf.matmul(hidden_layer3, weights_4) + biases_4),drop_keep)
  logits = tf.matmul(hidden_layer4, weights_o) + biases_o
  regu = 0.005 *(tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(weights_3) + tf.nn.l2_loss(weights_4) + tf.nn.l2_loss(weights_o)) 
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + regu)
  
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.023, global_step, 1, 0.9999)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step = global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
    
  valid_hidden1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1)
  valid_hidden2 = tf.nn.relu(tf.matmul(valid_hidden1, weights_2) + biases_2)
  valid_hidden3 = tf.nn.relu(tf.matmul(valid_hidden2, weights_3) + biases_3)
  valid_hidden4 = tf.nn.relu(tf.matmul(valid_hidden3, weights_4) + biases_4)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden4, weights_o) + biases_o)

  test_hidden1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
  test_hidden2 = tf.nn.relu(tf.matmul(test_hidden1, weights_2) + biases_2)
  test_hidden3 = tf.nn.relu(tf.matmul(test_hidden2, weights_3) + biases_3)
  test_hidden4 = tf.nn.relu(tf.matmul(test_hidden3, weights_4) + biases_4)
  test_prediction = tf.nn.softmax(tf.matmul(test_hidden4, weights_o) + biases_o)

In [188]:
import matplotlib.pyplot as plt
num_steps = 30001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
    
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % ((num_steps-1)/10) == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
    
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 3129.490723
Minibatch accuracy: 6.2%
Validation accuracy: 10.1%
Minibatch loss at step 3000: 1716.914429
Minibatch accuracy: 87.5%
Validation accuracy: 86.9%
Minibatch loss at step 6000: 1104.113647
Minibatch accuracy: 88.3%
Validation accuracy: 87.8%
Minibatch loss at step 9000: 796.206055
Minibatch accuracy: 89.1%
Validation accuracy: 88.6%
Minibatch loss at step 12000: 624.958435
Minibatch accuracy: 90.6%
Validation accuracy: 88.8%
Minibatch loss at step 15000: 522.318604
Minibatch accuracy: 92.2%
Validation accuracy: 89.2%
Minibatch loss at step 18000: 457.508148
Minibatch accuracy: 85.2%
Validation accuracy: 89.5%
Minibatch loss at step 21000: 414.514038
Minibatch accuracy: 89.8%
Validation accuracy: 89.3%
Minibatch loss at step 24000: 385.282867
Minibatch accuracy: 94.5%
Validation accuracy: 89.6%
Minibatch loss at step 27000: 364.989136
Minibatch accuracy: 96.1%
Validation accuracy: 89.8%
Minibatch loss at step 30000: 350.855652
Minibatch ac